# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
import plotly.express as px

cf.go_offline()

In [ ]:
data = pd.read_excel('../data/Online Retail.xlsx')

In [ ]:

# data_canada = px.data.gapminder().query("country == 'Canada'")
# fig = px.bar(data_canada, x='year', y='pop')
# fig.show()

data.head()

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [ ]:
brexit = data[(data['Country'] != 'United Kingdom') & (data['InvoiceDate'].astype(str).str.contains('^20.{2}-04-*', regex = True))]
brexit.head()

In [ ]:
brexitCountry = brexit.groupby('Country').agg({'Quantity':'sum', 'Revenue': 'sum'})

In [ ]:
brexitCountry.iplot(kind='bar', filename='cufflinks/categorical-bar-chart')


## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [ ]:
frenchbags = data[(data['Country'] == 'France') & (data['InvoiceDate'] >= '01-01-2011') & (data['InvoiceDate'] <= '31-05-2011')]
frenchbags.head()

In [ ]:
frenchgrouped = frenchbags.groupby('InvoiceDate').agg({'Quantity':'sum', 'Revenue': 'sum'})
frenchgrouped.head()
fig = frenchgrouped.iplot(filename='cufflinks/line-example')

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [ ]:
bunting = data[data['Description'] == 'PARTY BUNTING']
bunting.head()

In [ ]:
buntingbyCountry = bunting.groupby('Country').agg({'Quantity':'mean', 'UnitPrice': 'mean'})
buntingbyCountry.head()

In [ ]:
buntfig = buntingbyCountry.iplot(kind='scatter', 
                                 xTitle='Quantity', 
                                 yTitle='Unit Price', 
                                 mode='markers', 
                                 x='Quantity', 
                                 y='UnitPrice', 
                                 title='Mean quantity vs. mean Unitprice, by Country', 
                                 filename='cufflinks/simple-scatter')
# buntfig.update_layout(title_text='Your Mom') Esto ultimo no me va, no se porque

In [ ]:
buntingbyCountry.reset_index(inplace=True)

In [ ]:

fig = px.scatter(buntingbyCountry, x='Quantity', y="UnitPrice", color="Country",
                 )
fig.show()

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [ ]:
fewcountries = data[data['Country'].isin(['EIRE', 'Germany', 'France', 'Netherlands'])]

In [ ]:
fewcountries.head()

In [ ]:

quantityperinvoice = fewcountries.pivot_table(values='Quantity', columns='Country', 
                      index='InvoiceNo', aggfunc='sum')
quantityperinvoice.head()

In [ ]:
quantityperinvoice.iplot(kind='hist', xTitle='Quantity of Invoice', 
           yTitle='Amount of invoices', title='Distribution of invoice quantities')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [ ]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [ ]:
select = data[(data['Country'].isin(country_list)) & (data['Description'].isin(product_list))]
select.head()

In [ ]:
revbyProd = select.pivot_table(values='Revenue', 
                               columns='Country', 
                               index='Description', 
                               aggfunc='sum')
revbyProd

In [ ]:
revbyProd.iplot(kind='bar', 
                xTitle='Product', 
                yTitle='Revenue', 
                title='Revenue per product, by country', 
                filename='cufflinks/categorical-bar-chart')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [ ]:
uk.head()

In [ ]:
ukpivot = uk.pivot_table(values='Quantity', 
               columns=['Month','Year'], 
               index='Day', 
               aggfunc='sum').reset_index()
ukpivot.head()

In [ ]:
ukpivot.iplot(kind='line', x='Day')

In [ ]:
@interact(Selection=['Year', 'Month'])

def linechart(Selection):
    dats = uk.pivot_table(values='Quantity', columns=Selection,
                            index='Day', aggfunc='mean').reset_index()

    dats.iplot(kind='line', x='Day')

# Unfortunately the dropdown menu does not show up...

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [ ]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.